# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gemini-2.0-flash"
else:
    llm_model = "ggemini-2.5-flash"

In [4]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = ChatGoogleGenerativeAI(temperature=0.9, model=llm_model)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/c9/h7x3gp7x57n7kz8n91yjhw6w0000gn/T/ipykernel_49219/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "Queen Size Sheet Set"
chain.run(product)

/var/folders/c9/h7x3gp7x57n7kz8n91yjhw6w0000gn/T/ipykernel_49219/550859008.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Here are some name ideas, categorized for different styles and focuses:\n\n**Simple & Direct:**\n\n*   Queen Sheets Co.\n*   Queen Size Sheets\n*   The Queen Sheet Set\n*   Queen Bed Linens\n*   Perfect Queen Sheets\n\n**Focusing on Quality & Comfort:**\n\n*   Royal Comfort Sheets\n*   Queen's Embrace Linens\n*   Luxury Queen Sheets\n*   Dream Weaver Sheets (for Queen Size)\n*   Serene Sleep Queen\n*   The Cozy Queen\n\n**Modern & Evocative:**\n\n*   Queen & Co.\n*   Anya Sheets (if you want a personal, approachable feel)\n*   The Queen Edit\n*   Lunar Linens (evokes restful sleep)\n*   Cloud Nine Queen\n*   Ember Sheets (warm and inviting)\n\n**Unique & Playful:**\n\n*   Queen Bee Linens\n*   The Queen's Chamber\n*   Goodnight Queen\n*   Queen's Bounty\n*   Slumber Queen\n\n**Focusing on Material (if applicable):**\n\n*   Cotton Queen Sheets (if specializing in cotton)\n*   Linen Queen (if specializing in linen)\n*   Silk Queen Sheets (if specializing in silk)\n*   Bamboo Queen (if s

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatGoogleGenerativeAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some name ideas for a company that makes Queen Size Sheet Sets, broken down by category and with explanations:

**Simple & Direct:**

*   **Queen Sheets Co.** - Straightforward and easy to remember.
*   **Queen Size Linens** - Clearly states what you offer.
*   **Queen Comfort Sheets** - Focuses on the benefit of comfortable sheets.
*   **Perfect Queen Sheets** - Implies quality and fit.
*   **The Queen Sheet Set** - Simple, direct and emphasizes quality

**Emphasis on Comfort/Luxury:**

*   **Queen Dream Sheets** - Evokes a pleasant sleeping experience.
*   **Silken Queen** - Implies luxurious, silky smooth sheets.
*   **Queen Serenity Linens** - Focuses on relaxation and peace.
*   **Queen Cloud Comfort** - Suggests soft and fluffy sheets.
*   **The Regal Queen** - Luxury and quality

**Modern & Catchy:**

*   **Q.Sheets** - Short, modern, and memorable.
*   **The Sheet Lab - Queen** - Implies innovation and quality testing.
* 

"**Queen Size Sheet Sets: Luxurious comfort meets quality materials. Experience a regal night's sleep with our perfectly crafted bedding.**"

## SequentialChain

In [18]:
from langchain.chains import SequentialChain

In [19]:
llm = ChatGoogleGenerativeAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [20]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [21]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [22]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [23]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","language","followup_message"],
    verbose=True
)

In [24]:
review = df.Review[5]
overall_chain(review)

/var/folders/c9/h7x3gp7x57n7kz8n91yjhw6w0000gn/T/ipykernel_49219/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "This reviewer found the product's taste mediocre and the foam subpar, suggesting it might be an old batch or counterfeit compared to store-bought versions.",
 'language': 'The review is in **French**.',
 'followup_message': "Okay, here are a few follow-up responses to the French summary, ranging from empathetic to slightly confrontational, depending on the tone you want to convey:\n\n**Option 1: Empathetic and seeking more information (polite):**\n\n> Merci pour votre commentaire. Nous sommes désolés d'apprendre que votre expérience n'a pas été à la hauteur de vos attentes.  Pourriez-vous nous indiquer la date de péremption d

## Router Chain

In [55]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [26]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [27]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [28]:
llm = ChatGoogleGenerativeAI(temperature=0, model=llm_model)

In [29]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [30]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [49]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [50]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [51]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Alright, let's talk about black body radiation.\n\nImagine an object that absorbs *all* electromagnetic radiation that hits it, regardless of frequency or angle. That's a black body – a perfect absorber. Now, because it's absorbing energy, it also has to emit energy to stay in thermal equilibrium (otherwise it would just keep heating up forever!).  The radiation it emits is what we call black body radiation.\n\nHere's the key: the spectrum of light emitted by a black body depends *only* on its temperature.  It doesn't matter what the black body is made of, its shape, or anything else.  Just the temperature.\n\nThink of it like this:\n\n*   **Hotter black bodies:** Emit more radiation overall, and the peak of their emission shifts towards shorter wavelengths (bluer light).\n*   **Cooler black bodies:** Emit less radiation overall, and the peak of their emission shifts towards longer wavelengths (redder light).\n\nThis relationship is described mathematically by Planck's Law, which give

In [52]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Okay, this is a classic! Let\'s break it down, even though it seems simple:\n\n*   **Component 1: Understanding the "+" symbol** The "+" symbol represents addition. Addition is a fundamental arithmetic operation that combines two or more quantities to find their total.\n\n*   **Component 2: Understanding the number "2"** The number "2" represents a quantity of two. It could be two apples, two fingers, two anything!\n\n*   **Component 3: The operation** We are asked to combine a quantity of "2" with another quantity of "2".\n\n*   **Component 4: Performing the addition** If we start with two items and add two more, we can count the total number of items.\n\n*   **Component 5: The result** Counting two items and then counting two more results in a total of four items.\n\nTherefore, 2 + 2 = 4'

In [53]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the **blueprint for life**. It holds the complete set of instructions needed to build and operate an organism. Here's a breakdown of why it's essential in every cell:\n\n*   **Genetic Information:** DNA contains the genetic code that determines all of our traits, from eye color to susceptibility to certain diseases. This information is crucial for the cell's function and survival.\n\n*   **Protein Synthesis:** DNA contains the instructions for making proteins. Proteins are the workhorses of the cell, carrying out a vast array of functions, including:\n    *   **Enzymes:** Catalyzing biochemical reactions.\n    *   **Structural components:** Providing shape and support.\n    *   **Hormones:** Signaling molecules.\n    *   **Antibodies:** Defending against pathogens.\n\n*   **Cellular Function:** DNA dictates how a cell should function, including its growth, division, and specialization. Different cells express different genes (sections

Reminder: Download your notebook to you local computer to save your work.